In [ ]:
import awkward
from coffea import nanoevents
events = nanoevents\
         .NanoEventsFactory\
         .from_root('https://github.com/CoffeaTeam/coffea/blob/master/tests/samples/nano_dy.root?raw=true')\
         .events()
muons = events.Muon

In [ ]:
from abc import abstractmethod
from functools import partial
from copy import copy
import re

behavior = {}

@awkward.mixin_class(behavior)
class Systematic:
    """ A base class to describe and build variations on a feature of an nanoevents object. """

    def _ensure_systematics(self):
        if '__systematics__' not in awkward.fields(self):
            self['__systematics__'] = {}
    
    @property
    def systematics(self):
        regex = re.compile(r'\_{2}.*\_{2}')
        self._ensure_systematics()
        fields = [f for f in awkward.fields(self["__systematics__"]) if not regex.match(f)]
        return self["__systematics__"][fields]
        
    @abstractmethod
    def _build_variations(self, name, what, varying_function, *args, **kwargs):
        # name: str, name of the systematic variation / uncertainty source
        # what: Union[str, List[str], Tuple[str]], name what gets varied, 
        #       this could be a list or tuple of column names
        # varying_function: Union[function, bound method], a function that describes how 'what' is varied
        # *args: positional arguments to 'varying_function'
        # **kwargs: keyword arguments to 'varying function'
        # define how to manipulate the output of varying_function to produce
        # all systematic variations
        pass
    
    @abstractmethod
    def explodes_how(self):
        # this function contains decades of thinking about iterate over systematics variations
        # your opinions about systematics go here. :D
        pass
    
    @abstractmethod
    def describe_variations(self):
        pass
    
    def add_systematic(self, name, what, varying_function, *args, **kwargs):        
        # name: str, name of the systematic variation / uncertainty source
        # what: Union[str, List[str], Tuple[str]], name what gets varied, 
        #       this could be a list or tuple of column names
        # varying_function: Union[function, bound method], a function that describes how 'what' is varied
        # *args: positional arguments to 'varying_function'
        # **kwargs: keyword arguments to 'varying function'
        self._ensure_systematics()

        wrap = partial(awkward_rewrap, like_what=self["__systematics__"], gfunc=rewrap_recordarray)
        flat = awkward.flatten(self)
        
        if name in awkward.fields(flat["__systematics__"]):
            raise Exception(f"{name} already exists as a systematic for this object!")
        
        flat._build_variations(name, what, varying_function, *args, **kwargs)
        _, variations = self.describe_variations()
                
        flat["__systematics__", name] = awkward.zip(
            {v: getattr(flat, v)(name, what) for v in variations},
            depth_limit=1,
            with_name=f"{name}Systematics",
        )
        
        self["__systematics__"] = wrap(flat["__systematics__"])
        self.behavior[("__typestr__", f"{name}Systematics")] = f"{name}Systematics"
        

# we're gonna assume that the first record array we encounter is the flattened data
def rewrap_recordarray(layout, depth, data):
    if isinstance(layout, awkward.layout.RecordArray):
        return lambda: data
    return None        

def awkward_rewrap(arr, like_what, gfunc):
    behavior = awkward._util.behaviorof(like_what)
    func = partial(gfunc, data=arr.layout)
    layout = awkward.operations.convert.to_layout(like_what)
    newlayout = awkward._util.recursively_apply(layout, func)
    return awkward._util.wrap(newlayout, behavior=behavior)

@awkward.mixin_class(behavior)
class MuonUpDownSystematic(Systematic, nanoevents.methods.nanoaod.Muon):
    """ An example instance of a simple systematic with only up/down variations. """

    def _build_variations(self, name, what, varying_function, *args, **kwargs):
        whatarray = self[what]
        
        self["__systematics__", f"__{name}__"] = awkward.virtual(
            varying_function,
            args=(whatarray, *args),
            kwargs=kwargs,
            length=len(whatarray),
        )

    def describe_variations(self):
        return "Muon", ["up", "down"]
    
    def get_variation(self, name, what, updown):
        fields = awkward.fields(self)
        fields.remove("__systematics__")
        
        udmap = {"up": 0 , "down": 1}
        
        the_type, variations = self.describe_variations()
        
        varied = self["__systematics__", f"__{name}__", :, udmap[updown]]
        
        params = copy(self.layout.parameters)
        params["variation"] = f"{name}-{what}-{updown}"
        
        return awkward.zip(
            {field: self[field] if field != what else varied for field in fields},
            depth_limit=1,
            parameters=params,
            behavior=self.behavior,
            with_name=the_type,
        )
    
    def up(self, name, what):
        return awkward.virtual(
            self.get_variation,
            args=(name, what, "up"),
            length=len(self),
        )
    
    def down(self, name, what):
        return awkward.virtual(
            self.get_variation,
            args=(name, what, "down"),
            length=len(self),
            parameters=self[what].layout.parameters,
        )

behavior[("__typestr__", "Systematic")] = "Systematic"
behavior[("__typestr__", "MuonUpDownSystematic")] = "MuonUpDownSystematic"    

In [ ]:
awkward.behavior.update(behavior)

# Right now this is a little backwards from where I would like it to be.
# In particular, I'd like to be able to .add_systematic() and specify the 
# systematic type then, instead of having to do an early and static binding
# as we have here. This should be possible.

syst_muons = awkward.with_name(muons, "MuonUpDownSystematic")

import numpy as np
def muon_pt_scale(pt):   
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * pt[:, None]

def muon_pt_resolution(pt):   
    return np.random.normal(pt[:,None], np.array([0.01, 0.02], dtype=np.float32))

syst_muons.add_systematic("PtScale", "pt", muon_pt_scale)
syst_muons.add_systematic("PtResolution", "pt", muon_pt_resolution)


In [ ]:
syst_muons.pt

In [ ]:
syst_muons.systematics.PtScale.up.pt

In [ ]:
syst_muons.systematics.PtScale.down.pt

In [ ]:
# TODO: Make it so that syst_muons.pt > X returns boolean values
#       for all variations over X. 
#       Requires some tracking of (pieces of) "what".